## FORMULA 1

- I attempt to apply machine learning alogirthms to a few Formula 1 scenarios
    - a) Binary Classification: Predict race finishes (ie. Did a driver finish a race?)
    - b) Binary Classifiation (of multiple target variables): Predict tyre set combinations each driver will use at each race.
    - c) Multi-label Classifiation: Predict tyre set combinations each driver will use at each race. 
    
- Early challenges in starting this project was that fact that I do not have a prepared full dataset of features. I had to create a dataset myself. This involves creating a template of race participants for the 2015 to 2017 season, then merging features to this template. Furthermore, all features were engineered.

- Next challenge was the fact that i had to deal with 20 races per season. This means that the training dataset is built up as the season progresses. This increases variability of prediction estimation. Furthermore, it makes code execution (particulatly tuning of classifiers) computationally more expensive. 

- Section B to D of the notebook is filled with code of all functions requried to create dataset and then conduct model-fitting and generating predictions. (I have made the functions as 'recyclable' as possible for future seasons). Section E is where the meat lies: the prediction results. 

- As i did not want to bloat this notebook, I have placed all data visualizations (EDA, prediction results) in a separate notebook titled "FORMULA 1: DATA VISUALIZATION"

In [1]:
import numpy as np
import scipy 
import scipy.stats
from scipy import stats
import seaborn as sns
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import itertools
from itertools import groupby
import pickle
import os
import math
from sympy import S, symbols
from collections import Counter
import sklearn 
from sklearn import preprocessing

pd.options.mode.chained_assignment = None 

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

sns.set(style='white', context='notebook', palette='deep')
#sns.mpl.rcParams['figure.figsize'] = (16, 10)

# Directory to store pickled dataframes
directory = '/Users/dianaow/Documents/formula-1-race-data/dataframes/'

In [2]:
import statsmodels.api as sm
from sklearn.metrics import r2_score , mean_absolute_error, mean_squared_error
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.utils import resample
from sklearn.utils import class_weight

from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve, train_test_split
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, accuracy_score, precision_score, average_precision_score, \
classification_report, recall_score, confusion_matrix, f1_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold 
from sklearn.base import clone

from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.under_sampling import TomekLinks

import statsmodels.graphics.regressionplots 
import mord as mord
from skmultilearn.problem_transform import ClassifierChain

/Users/dianaow/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
import warnings
warnings.filterwarnings(action='ignore')

In [4]:
def read_from_pickle(directory, filename):
    df = pd.DataFrame()
    filepath = directory + filename
    with open(filepath, 'rb') as file:
        df = pickle.load(file)
            
    return df

## Section A: Create template to build dataset of features (predictors)

#### Word of caution before building dataset:

- Race list is constant between 2016 and 2017.
- Driver list is not constant between 2016 and 2017.
- Retired: Rosberg, Button
- New in 2017: Stroll, Vandoorne (participated in one race in 2016), Ocon (participated one race in 2016), Hulkenberg, Wehrlein
- No longer participating in 2017: Haryanto, Nasr, Gutierrez
- Mid-season team switches.
    - In 2017, Sainz switched from Torro Rosso to Renault starting from US GP.
    - In 2016, Kvyat switched from Torro Rosso to Red Bull starting from Spanish GP.
    - In 2016, Verstappen switched from Red Bull to Torro Rosso starting from Spanish GP.
- Dropped mid-season. 
    - Haryanto (2016), Kvyat (2017), Palmer (2017)
- Added mid-season. 
    - Ocon (2016, from Belgian GP), Gasly (2017, from Malaysian GP), Hartley (2017, from US GP)
- Reserve drivers who substituted:
    - 2016 Bahrain GP -> Vandoorne subbed for Alonso
    - 2017 Monaco GP -> Button subbed for Alonso
    - 2017 Hungarian GP -> Di Resta subbed for Massa

These driver switches will causes some confusion in putting together the final dataset. Always check! 

In [5]:
ALL151617_startstint = read_from_pickle(directory, "ALL151617_startstint.pickle")
df_drivers_2015 = read_from_pickle(directory, "df_drivers_2015.pickle")
df_drivers_2016 = read_from_pickle(directory, "df_drivers_2016.pickle")
df_drivers_2017 = read_from_pickle(directory, "df_drivers_2017.pickle")
df_races = read_from_pickle(directory, "df_races.pickle")

In [6]:
df_results = pd.read_csv('./formula-1-race-data/results.csv')

In [7]:
drivers_2015 = list(df_drivers_2015.driverRef.unique()) 
drivers_2015.remove('magnussen') # remove magnussen because he only participated in 1 race in 2016
# both drivers didn't complete the entire season. since manor is no longer an existing team, remove these drivers
drivers_2015.remove('rossi') 
drivers_2015.remove('merhi')
#drivers_2015.remove('stevens') 

drivers_2016 = list(df_drivers_2016.driverRef.unique()) 
drivers_2016.remove('vandoorne') # remove vandoorne because he only participated in 1 race in 2016
drivers_2016.remove('haryanto')
drivers_2016.remove('ocon')

drivers_2017 = list(df_drivers_2017.driverRef.unique())
# remove giovinazzi, gasly, hartley, button, di resta because they only participated in races one-off in 2017
drivers_2017.remove('button')
drivers_2017.remove('giovinazzi')
drivers_2017.remove('di resta')
drivers_2017.remove('palmer')
drivers_2017.remove('gasly')
drivers_2017.remove('kvyat')
drivers_2017.remove('brendon_hartley')

# Standardize: Use the same number of races in 2017
races15 = list(df_races[df_races['year'] == 2015].name.unique())
races = list(df_races[df_races['year'] == 2017].name.unique())
stints = range(1,5)

template15 = pd.DataFrame(list(itertools.product([2015], races15, drivers_2015)), columns=['year', 'name', 'driverRef'])
template16 = pd.DataFrame(list(itertools.product([2016], races, drivers_2016)), columns=['year', 'name', 'driverRef'])
template17 = pd.DataFrame(list(itertools.product([2017], races, drivers_2017)), columns=['year', 'name', 'driverRef'])

drivers_2017_new = ['gasly', 'kvyat', 'brendon_hartley']

temp_har = pd.DataFrame(list(itertools.product([2016], ['Australian Grand Prix',
                                                         'Chinese Grand Prix',
                                                         'Bahrain Grand Prix',
                                                         'Russian Grand Prix',
                                                         'Spanish Grand Prix',
                                                         'Monaco Grand Prix',
                                                         'Canadian Grand Prix',
                                                         'Azerbaijan Grand Prix',
                                                         'Austrian Grand Prix',
                                                         'British Grand Prix',
                                                         'Hungarian Grand Prix'] ,\
                        ["haryanto"])), columns=['year', 'name', 'driverRef'])

temp_ocon = pd.DataFrame(list(itertools.product([2016], ['Belgian Grand Prix',
                                                         'Italian Grand Prix',
                                                         'Singapore Grand Prix',
                                                         'Malaysian Grand Prix',
                                                         'Japanese Grand Prix',
                                                         'United States Grand Prix',
                                                         'Mexican Grand Prix',
                                                         'Brazilian Grand Prix',
                                                         'Abu Dhabi Grand Prix'],\
                        ["ocon"])), columns=['year', 'name', 'driverRef'])

temp_gasly = pd.DataFrame(list(itertools.product([2017], ['Malaysian Grand Prix',
                                                         'Japanese Grand Prix',
                                                         'Mexican Grand Prix',
                                                         'Brazilian Grand Prix',
                                                         'Abu Dhabi Grand Prix'],\
                        ["gasly"])), columns=['year', 'name', 'driverRef'])

temp_hartley = pd.DataFrame(list(itertools.product([2017], ['United States Grand Prix',
                                                             'Mexican Grand Prix',
                                                             'Brazilian Grand Prix',
                                                             'Abu Dhabi Grand Prix'],\
                        ["brendon_hartley"])), columns=['year', 'name', 'driverRef'])

temp_kvyat = pd.DataFrame(list(itertools.product([2017], ['Australian Grand Prix',
                                                             'Chinese Grand Prix',
                                                             'Bahrain Grand Prix',
                                                             'Russian Grand Prix',
                                                             'Spanish Grand Prix',
                                                             'Monaco Grand Prix',
                                                             'Canadian Grand Prix',
                                                             'Azerbaijan Grand Prix',
                                                             'Austrian Grand Prix',
                                                             'British Grand Prix',
                                                             'Hungarian Grand Prix',
                                                             'Belgian Grand Prix',
                                                             'Italian Grand Prix',
                                                             'Singapore Grand Prix',
                                                             'United States Grand Prix'] ,\
                        ["kvyat"])), columns=['year', 'name', 'driverRef'])


temp_palmer = pd.DataFrame(list(itertools.product([2017], ['Australian Grand Prix',
                                                             'Chinese Grand Prix',
                                                             'Bahrain Grand Prix',
                                                             'Russian Grand Prix',
                                                             'Spanish Grand Prix',
                                                             'Monaco Grand Prix',
                                                             'Canadian Grand Prix',
                                                             'Azerbaijan Grand Prix',
                                                             'Austrian Grand Prix',
                                                             'British Grand Prix',
                                                             'Hungarian Grand Prix',
                                                             'Belgian Grand Prix',
                                                             'Italian Grand Prix',
                                                             'Singapore Grand Prix',
                                                             'Malaysian Grand Prix'] ,\
                        ["palmer"])), columns=['year', 'name', 'driverRef'])
                                                               
                                                               
        
# Create a new dataframe with the actual race information of these replacement/one-off drivers

from itertools import product

replacements = pd.DataFrame({'year': [2016, 2017, 2017, 2017, 2017], 
                             'name': ["Bahrain Grand Prix", "Australian Grand Prix", "Chinese Grand Prix", "Monaco Grand Prix", "Hungarian Grand Prix"], 
                             'driverRef': ['vandoorne', 'giovinazzi', 'giovinazzi', "button", "di resta"]
                             })

template = pd.concat([template15, template16, template17, temp_har, temp_ocon, temp_gasly, temp_hartley, temp_kvyat, temp_palmer])

In [8]:
# Merge constructor Ids to template
dcrm = read_from_pickle(directory, "dcrm.pickle")
template_dcrm = pd.merge(template, dcrm[['year', 'name', 'driverRef', 'constructorRef', 'constructorId']], on=['year', 'name', 'driverRef'], how='left')

# Note there are null values because these drivers did not participate in the respective races for once or twice only. 
# Inpute their constructordIds
template_dcrm[template_dcrm['constructorRef'].isnull()].drop_duplicates(subset=['year', 'name', 'driverRef'], keep='first')

,year,name,driverRef,constructorRef,constructorId
112,2015,Australian Grand Prix,alonso,NaN,NaN
113,2015,Australian Grand Prix,stevens,NaN,NaN
422,2016,Bahrain Grand Prix,alonso,NaN,NaN
798,2017,Australian Grand Prix,wehrlein,NaN,NaN
816,2017,Chinese Grand Prix,wehrlein,NaN,NaN
882,2017,Monaco Grand Prix,alonso,NaN,NaN
966,2017,Hungarian Grand Prix,massa,NaN,NaN


In [9]:
template_dcrm.loc[112,"constructorRef"] = 'mclaren'
template_dcrm.loc[113,"constructorRef"] = 'manor'
template_dcrm.loc[422,"constructorRef"] = 'mclaren'
template_dcrm.loc[798, "constructorRef"] = 'sauber'
template_dcrm.loc[816, "constructorRef"] = 'sauber'
template_dcrm.loc[882, "constructorRef"] = 'mclaren'
template_dcrm.loc[966,"constructorRef"] = 'williams'

template_dcrm.loc[112,"constructorId"] = 1
template_dcrm.loc[113,"constructorId"] = 209
template_dcrm.loc[422,"constructorId"] = 1
template_dcrm.loc[798, "constructorId"] = 15
template_dcrm.loc[816, "constructorId"] = 15
template_dcrm.loc[882, "constructorId"] = 1
template_dcrm.loc[966,"constructorId"] = 3

template_dcrm.replace("brendon_hartley", "hartley", inplace=True)

In [10]:
template17 = template_dcrm[template_dcrm['year'] == 2017]

### Subset dataset template

#### Pirelli chooses tyre compounds for each track. No less than nine weeks before the start of each event in Europe, and 15 weeks before the start of each event held outside Europe, Pirelli will inform teams which three compounds are to be used at each race. A race may not be allocated the same combination for consecutive seasons. The choice of tyre compound combinations is important in shaping race strategy.

#### Pirelli's possible tyre combinations for the 2017 session are as follows:
- Super soft, Soft, Medium -> Category 1
- Soft, Medium, Hard -> Category 2
- Super soft, Soft, Ultra soft -> Category 3

In [11]:
xl = pd.ExcelFile("/Users/dianaow/Downloads/F1_Tyre_data.xlsx")
xl.sheet_names
pirelli = xl.parse("Sheet7")

In [12]:
# Pirelli dataframe: Dataframe containing Pirelli's allocated mandatory tyre set for each race for 2015, 2016, 2017 season
# Label each selected set
# SS, S, M = 1
# S, M, H = 2
# SS, S, US = 3

for i,row in pirelli.iterrows(): 
    if (pirelli.loc[i,'Super Soft'] == 'Super Soft') & (pirelli.loc[i,'Soft'] == 'Soft') & (pirelli.loc[i,'Medium'] == "Medium"):  
        pirelli.loc[i,'mandatory combi'] = float(1)
    elif (pirelli.loc[i,'Soft'] == "Soft") & (pirelli.loc[i,'Medium'] == "Medium") & (pirelli.loc[i,'Hard'] == "Hard"):
        pirelli.loc[i,'mandatory combi'] = float(2)
    elif (pirelli.loc[i,'Super Soft'] == 'Super Soft') & (pirelli.loc[i,'Soft'] == 'Soft') & (pirelli.loc[i,'Ultra Soft'] == 'Ultra Soft'):
        pirelli.loc[i,'mandatory combi'] = float(3)
    elif (pirelli.loc[i,'Super Soft'] == 'Super Soft') & (pirelli.loc[i,'Soft'] == 'Soft') & (pirelli.loc[i,'Medium'] != "Medium"): 
        pirelli.loc[i,'mandatory combi'] = float(4)
    elif (pirelli.loc[i,'Super Soft'] != 'Super Soft') & (pirelli.loc[i,'Soft'] == 'Soft') & (pirelli.loc[i,'Medium'] != "Medium"): 
        pirelli.loc[i,'mandatory combi'] = float(5)
    elif (pirelli.loc[i,'Super Soft'] != 'Soft') & (pirelli.loc[i,'Medium'] == "Medium") & (pirelli.loc[i,'Hard'] == "Hard"): 
        pirelli.loc[i,'mandatory combi'] = float(6)
    else:
        pirelli.loc[i,'mandatory combi'] = float(7)

#### Races allocated Combination 1

In [13]:
p1 = pirelli[pirelli['mandatory combi'] == 1].sort_values(['name', 'year'])
p1_2017_races = list(p1[p1['year'] == 2017].name.unique()) # Races are sorted in alphabetical order
p1_2017_races.sort(key=lambda x: races.index(x)) # Sort races according to the race calendar!
p1_races = p1[['year', 'name']].to_dict('list')
p1
# Note: It is not always that a race will be allocated the same tyre combination between years.
# For eg. 2016 Brazilian GP: combi 2, 2017 Brazilian GP: combi 1

,year,name,Super Soft,Soft,Medium,Hard,Ultra Soft,mandatory combi
0,2016,Australian Grand Prix,Super Soft,Soft,Medium,NaN,NaN,1.0
7,2016,Azerbaijan Grand Prix,Super Soft,Soft,Medium,NaN,NaN,1.0
47,2017,Azerbaijan Grand Prix,Super Soft,Soft,Medium,NaN,NaN,1.0
1,2016,Bahrain Grand Prix,Super Soft,Soft,Medium,NaN,NaN,1.0
42,2017,Bahrain Grand Prix,Super Soft,Soft,Medium,NaN,NaN,1.0
13,2016,Belgian Grand Prix,Super Soft,Soft,Medium,NaN,NaN,1.0
58,2017,Brazilian Grand Prix,Super Soft,Soft,Medium,NaN,NaN,1.0
49,2017,British Grand Prix,Super Soft,Soft,Medium,NaN,NaN,1.0
2,2016,Chinese Grand Prix,Super Soft,Soft,Medium,NaN,NaN,1.0
41,2017,Chinese Grand Prix,Super Soft,Soft,Medium,NaN,NaN,1.0


#### Races allocated Combination 2

In [14]:
p2 = pirelli[pirelli['mandatory combi'] == 2].sort_values(['name', 'year'])
p2_2017_races = list(p2[p2['year'] == 2017].name.unique())
p2_2017_races.sort(key=lambda x: races.index(x)) # Sort races according to the race calendar!
p2_races = p2[['year', 'name']].to_dict('list')
p2

,year,name,Super Soft,Soft,Medium,Hard,Ultra Soft,mandatory combi
19,2016,Brazilian Grand Prix,NaN,Soft,Medium,Hard,NaN,2.0
9,2016,British Grand Prix,NaN,Soft,Medium,Hard,NaN,2.0
16,2016,Japanese Grand Prix,NaN,Soft,Medium,Hard,NaN,2.0
15,2016,Malaysian Grand Prix,NaN,Soft,Medium,Hard,NaN,2.0
4,2016,Spanish Grand Prix,NaN,Soft,Medium,Hard,NaN,2.0
44,2017,Spanish Grand Prix,NaN,Soft,Medium,Hard,NaN,2.0


#### Races allocated Combination 3

In [15]:
p3 = pirelli[pirelli['mandatory combi'] == 3].sort_values(['name', 'year'])
p3_2017_races = list(p3[p3['year'] == 2017].name.unique())
p3_2017_races.sort(key=lambda x: races.index(x)) # Sort races according to the race calendar!
p3_races = p3[['year', 'name']].to_dict('list')
p3

,year,name,Super Soft,Soft,Medium,Hard,Ultra Soft,mandatory combi
20,2016,Abu Dhabi Grand Prix,Super Soft,Soft,NaN,NaN,Ultra Soft,3.0
59,2017,Abu Dhabi Grand Prix,Super Soft,Soft,NaN,NaN,Ultra Soft,3.0
40,2017,Australian Grand Prix,Super Soft,Soft,NaN,NaN,Ultra Soft,3.0
8,2016,Austrian Grand Prix,Super Soft,Soft,NaN,NaN,Ultra Soft,3.0
48,2017,Austrian Grand Prix,Super Soft,Soft,NaN,NaN,Ultra Soft,3.0
51,2017,Belgian Grand Prix,Super Soft,Soft,NaN,NaN,Ultra Soft,3.0
6,2016,Canadian Grand Prix,Super Soft,Soft,NaN,NaN,Ultra Soft,3.0
46,2017,Canadian Grand Prix,Super Soft,Soft,NaN,NaN,Ultra Soft,3.0
57,2017,Mexican Grand Prix,Super Soft,Soft,NaN,NaN,Ultra Soft,3.0
5,2016,Monaco Grand Prix,Super Soft,Soft,NaN,NaN,Ultra Soft,3.0


In [108]:
races_dict = pirelli[['year', 'name']].to_dict('list')

In [17]:
p1.to_pickle(os.path.join(directory,  "p1.pickle"))
p2.to_pickle(os.path.join(directory,  "p2.pickle"))
p3.to_pickle(os.path.join(directory,  "p3.pickle"))
pirelli.to_pickle(os.path.join(directory,  "pirelli.pickle"))

In [18]:
# List of drivers participating in 2015 to 2017 seasons

df_drivers = df_drivers_2015.append(df_drivers_2016)
df_drivers = df_drivers.append(df_drivers_2017)
df_drivers = df_drivers.drop_duplicates(subset=['driverId', 'driverRef'], keep='first')

In [19]:
# Race finish position numbers of drivers

finish_pos = df_results[['raceId', 'driverId', 'position']]
finish_pos = pd.merge(finish_pos, df_races[['raceId', "name", 'year']], on='raceId', how='left')
finish_pos = pd.merge(finish_pos, df_drivers, on='driverId', how='left')
finish_pos = finish_pos[finish_pos['year'].isin([2015,2016,2017])]
finish_pos.drop(['raceId', 'driverId'], axis=1, inplace=True)
finish_pos = finish_pos.rename(columns={'position': 'finish pos'}).reset_index(drop=True)
finish_pos.to_pickle(os.path.join(directory,  "finish_pos.pickle"))

# Section B: Features / Predictors

IMPORTANT: We can only include features that are known before a race. (Note: Qualification results can be a feature of the model if we plan to perform the classification after qualification and before a race.) 

PLEASE REFER TO JUPYTER NOTEBOOK TITLED 'FORMULA 1: DATA VISUALIZATION' FOR DETAILED VISUALIATION OF FEATURES!

## Features that are the ground truth

### 1) Feature: Team driver belongs to 

In [20]:
# Label teams before dummy coding

top_field = ['mercedes', 'ferrari', 'red_bull']
mid_field = ['haas', 'toro_rosso', 'mclaren', 'williams', 'force_india']
bottom_field = ['manor', 'sauber', 'renault']

teams_cat_top = dict((i, "top") for i in top_field)
teams_cat_mid = dict((i, "middle") for i in mid_field)
teams_cat_bottom = dict((i, "bottom") for i in bottom_field)

def merge_dicts(x, y, z):
    M = x.copy()   # start with x's keys and values
    M.update(y)    # modifies z with y's keys and values & returns None
    M.update(z)
    
    return M

team_cat = merge_dicts(teams_cat_top, teams_cat_mid, teams_cat_bottom)

In [21]:
template_dcrm['team category'] = template_dcrm['constructorRef'].map(team_cat)
template17['team category'] = template17['constructorRef'].map(team_cat)

In [22]:
team_cat_dummies = pd.get_dummies(template_dcrm, columns=["team category"], drop_first=True)

### 2) Feature: Driver's Qualification Position

In [23]:
df_qualifying = pd.read_csv('./formula-1-race-data/qualifying.csv')

qual_pos = df_qualifying[['raceId', 'driverId', 'position']]
qual_pos = pd.merge(qual_pos, df_races[['raceId', "name", 'year']], on='raceId', how='left')
qual_pos = pd.merge(qual_pos, df_drivers, on='driverId', how='left')
qual_pos = qual_pos[qual_pos['year'].isin([2015,2016,2017])]
qual_pos.drop(['raceId', 'driverId'], axis=1, inplace=True)
qual_pos.to_pickle(os.path.join(directory,  "qual_pos.pickle"))

In [24]:
qual_pos.head()

,position,name,year,driverRef
6286,1,Australian Grand Prix,2015,hamilton
6287,2,Australian Grand Prix,2015,rosberg
6288,3,Australian Grand Prix,2015,massa
6289,4,Australian Grand Prix,2015,vettel
6290,5,Australian Grand Prix,2015,raikkonen


### 3) Feature: Selected tyre sets of each driver for each race
- Published by Pirelli 2 weeks before each race
- Some 2016 or 2017 races do not have this information published by Pirelli online. -> 2016 Italian Grand Prix, 2016 Mexican Grand Prix, 2017 Australian Grand Prix, 2017 Bahrain Grand Prix, 2017 Russian Grand Prix, 2017 Chinese Grand Prix
- Approach 1: Inpute these values with the average count of tyres selected for each tyre type for races assigned the same mandatory tyre sets. 
- Approach 2:
    - If Pirelli's assigned tyre combination for the track is same for both 2016 and 2017, missing 2017 values will be inputed with the previous year's data. Missing 2016 values will be inputed with next year's data
    - If Pirelli's assigned tyre combination for the track is not the same for both 2016 and 2017, missing 2017 values will be inputed with the previous year's data in the order of the softness of the tyre compounds available
- Choose Approach 2 because the ratio of tyre compounds selected is track-specific

#### Approach 1

In [25]:
xl = pd.ExcelFile("./formula-1-race-data/Selected_Sets.xlsx", encoding='utf-8')
xl.sheet_names
selected_sets1 = xl.parse("Selected_Sets_Copy")

In [26]:
selected_sets1.head()

,year,name,driverRef,Medium,Soft,Super Soft,Ultra soft,Hard
0,2017,Brazilian Grand Prix,vettel,1,3,9,0,0
1,2017,Brazilian Grand Prix,bottas,1,4,8,0,0
2,2017,Brazilian Grand Prix,raikkonen,1,3,9,0,0
3,2017,Brazilian Grand Prix,hamilton,1,4,8,0,0
4,2017,Brazilian Grand Prix,verstappen,1,3,9,0,0


#### Approach 2

In [27]:
xl = pd.ExcelFile("./formula-1-race-data/Selected_Sets.xlsx", encoding='utf-8')
xl.sheet_names
selected_sets = xl.parse("Selected_Sets_Copy")

In [28]:
selected_sets['tyre combi'] = selected_sets[["Medium", "Soft", "Super Soft", "Ultra soft", "Hard"]].values.tolist()
selected_sets['tyre combi'] = [tuple(x) for x in selected_sets['tyre combi']]
selected_sets = selected_sets[['year', 'name', 'driverRef', 'tyre combi']]

In [29]:
selected_sets = pd.merge(selected_sets, pirelli[['year', 'name', 'mandatory combi']], on=['year', 'name'], how='left')

In [30]:
selected_sets[(selected_sets['name'] == "Spanish Grand Prix") & (selected_sets['year'] == 2017)]

,year,name,driverRef,tyre combi,mandatory combi
280,2017,Spanish Grand Prix,hamilton,"(1, 5, 7, 0, 0)",2.0
281,2017,Spanish Grand Prix,bottas,"(1, 5, 7, 0, 0)",2.0
282,2017,Spanish Grand Prix,ricciardo,"(2, 4, 7, 0, 0)",2.0
283,2017,Spanish Grand Prix,verstappen,"(2, 4, 7, 0, 0)",2.0
284,2017,Spanish Grand Prix,vettel,"(1, 4, 8, 0, 0)",2.0
285,2017,Spanish Grand Prix,raikkonen,"(1, 4, 8, 0, 0)",2.0
286,2017,Spanish Grand Prix,perez,"(2, 5, 6, 0, 0)",2.0
287,2017,Spanish Grand Prix,ocon,"(2, 5, 6, 0, 0)",2.0
288,2017,Spanish Grand Prix,massa,"(1, 5, 7, 0, 0)",2.0
289,2017,Spanish Grand Prix,stroll,"(1, 5, 7, 0, 0)",2.0


In [31]:
sets_group = selected_sets.groupby(['tyre combi']).agg({"driverRef":"count"}).reset_index()
sets_group['freq'] = sets_group['driverRef'] / len(selected_sets)
print len(sets_group)

sets_group_top95 = sets_group[sets_group['freq'] >= 0.05]
sets_group_top95

73


,tyre combi,driverRef,freq
1,"(0, 1, 2, 10, 0)",49,0.058333
22,"(1, 2, 10, 0, 0)",42,0.050000
23,"(1, 3, 9, 0, 0)",48,0.057143
24,"(1, 4, 8, 0, 0)",52,0.061905
26,"(1, 5, 7, 0, 0)",72,0.085714
35,"(2, 4, 7, 0, 0)",45,0.053571


In [32]:
sets_group_yn = selected_sets.groupby(['year', 'name', 'tyre combi']).agg({"driverRef":"count"}).reset_index()
sets_group_yn = pd.merge(sets_group_yn, pirelli[['year', 'name', 'mandatory combi']], on=['year', 'name'], how='left')

In [33]:
sets1 = sets_group_yn[sets_group_yn['mandatory combi'] == 1].reset_index(drop=True)
sets2 = sets_group_yn[sets_group_yn['mandatory combi'] == 2].reset_index(drop=True)
sets3 = sets_group_yn[sets_group_yn['mandatory combi'] == 3].reset_index(drop=True)

In [34]:
sets1_g = sets1.groupby('tyre combi').agg({"driverRef":"sum"}).reset_index()
sets1_g['freq'] = sets1_g['driverRef'] / sum(sets1_g['driverRef'])
sets1_g_top90 = sets1_g[sets1_g['freq'] >= 0.10]
sets1_g_top90

,tyre combi,driverRef,freq
1,"(1, 2, 10, 0, 0)",42,0.1050
2,"(1, 3, 9, 0, 0)",48,0.1200
3,"(1, 4, 8, 0, 0)",50,0.1250
4,"(1, 5, 7, 0, 0)",65,0.1625
12,"(2, 4, 7, 0, 0)",43,0.1075


In [35]:
sets2_g = sets2.groupby('tyre combi').agg({"driverRef":"sum"}).reset_index()
sets2_g['freq'] = sets2_g['driverRef'] / sum(sets2_g['driverRef'])
sets2_g_top90 = sets2_g[sets2_g['freq'] >= 0.10]
sets2_g_top90

,tyre combi,driverRef,freq
23,"(4, 7, 0, 0, 2)",14,0.107692
25,"(4, 8, 0, 0, 1)",13,0.100000
27,"(5, 7, 0, 0, 1)",24,0.184615


In [36]:
sets3_g = sets3.groupby('tyre combi').agg({"driverRef":"sum"}).reset_index()
sets3_g['freq'] = sets3_g['driverRef'] / sum(sets3_g['driverRef'])
sets3_g_top90 = sets3_g[sets3_g['freq'] >= 0.10]
sets3_g_top90

,tyre combi,driverRef,freq
1,"(0, 1, 2, 10, 0)",49,0.158065
2,"(0, 1, 3, 9, 0)",31,0.100000
9,"(0, 2, 4, 7, 0)",32,0.103226


In [37]:
cols_to_keep = ['year', 'name', 'driverRef', 'mandatory combi', 'tyre combi_(1, 2, 10, 0, 0)', 'tyre combi_(1, 3, 9, 0, 0)',\
                'tyre combi_(1, 4, 8, 0, 0)', 'tyre combi_(1, 5, 7, 0, 0)', 'tyre combi_(2, 4, 7, 0, 0)',\
                'tyre combi_(4, 7, 0, 0, 2)', 'tyre combi_(4, 8, 0, 0, 1)', 'tyre combi_(5, 7, 0, 0, 1)',\
                'tyre combi_(0, 1, 2, 10, 0)', 'tyre combi_(0, 1, 3, 9, 0)', 'tyre combi_(0, 2, 4, 7, 0)']

In [38]:
sets_dummies = pd.get_dummies(selected_sets, columns=["tyre combi"])

In [39]:
sets_dummies_trunc = sets_dummies[cols_to_keep]

In [40]:
selected_sets_all = pd.merge(selected_sets, sets_dummies, on=['year', 'name', 'driverRef', 'mandatory combi'], how='left')

In [41]:
mand_combi_dummies = pd.get_dummies(selected_sets, columns=["mandatory combi"], drop_first=True)

In [42]:
selected_sets_all = pd.merge(selected_sets_all, mand_combi_dummies, on=['year', 'name', 'driverRef', 'tyre combi'], how='left')
selected_sets_all.drop(['tyre combi','mandatory combi'], axis=1, inplace=True)

In [43]:
selected_sets_all.head()

,year,name,driverRef,"tyre combi_(0, 1, 1, 11, 0)","tyre combi_(0, 1, 2, 10, 0)","tyre combi_(0, 1, 3, 9, 0)","tyre combi_(0, 1, 4, 8, 0)","tyre combi_(0, 1, 5, 7, 0)","tyre combi_(0, 1, 6, 6, 0)","tyre combi_(0, 2, 1, 10, 0)",...,"tyre combi_(4, 7, 0, 2, 0)","tyre combi_(4, 8, 0, 0, 1)","tyre combi_(5, 2, 6, 0, 0)","tyre combi_(5, 4, 4, 0, 0)","tyre combi_(5, 6, 0, 0, 2)","tyre combi_(5, 7, 0, 0, 1)","tyre combi_(6, 5, 0, 0, 2)","tyre combi_(6, 6, 0, 0, 1)",mandatory combi_2.0,mandatory combi_3.0
0,2017,Brazilian Grand Prix,vettel,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2017,Brazilian Grand Prix,bottas,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2017,Brazilian Grand Prix,raikkonen,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2017,Brazilian Grand Prix,hamilton,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2017,Brazilian Grand Prix,verstappen,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Features derived from clustering

### 1) Feature: Races clustered based on track characteristics 

In [44]:
clusters = read_from_pickle(directory, "clusters.pickle")

In [45]:
clusters

,name,clusters
0,Abu Dhabi Grand Prix,0
1,Australian Grand Prix,2
2,Austrian Grand Prix,3
3,Azerbaijan Grand Prix,1
4,Bahrain Grand Prix,1
5,Belgian Grand Prix,2
6,Brazilian Grand Prix,0
7,British Grand Prix,2
8,Canadian Grand Prix,0
9,Chinese Grand Prix,3


## Features which we do not have the ground truth for pre-race

- These are features that relate to the target variable, but value are unknown before the race. If they are to be utilized, these values have to be estimated.

### Feature: Driver's race finish position

In [46]:
df_results_new = pd.merge(df_results, df_drivers[['driverId', 'driverRef']], on=['driverId'], how='left')
df_results_new = pd.merge(df_results_new, df_races[['raceId', 'year', 'name']], on=['raceId'], how='left')

# Feature: Finish position
df_results_new['position'] = df_results_new['position'].replace(range(1,4) ,"Podium")
df_results_new['position'] = df_results_new['position'].replace(range(5,10) , "Pos 4 to 10")
df_results_new['position'] = df_results_new['position'].replace(np.nan , "Did not finish")
mask = ~df_results_new['position'].isin(['Podium',"Pos 4 to 10", "Did not finish"])
df_results_new['position'] = df_results_new['position'].mask(mask, "Pos > 10")

### Feature: Reason category for not finishing race

- Finished / Accident/ Technical Failure (various reasons such as Brake Wear, Oil Leak etc are classified as Technical Failure)

In [47]:
# Reason category for not finishing race
df_results_new['statusId'] = df_results_new['statusId'].replace([1,11,12,13,14] ,"Finished")
df_results_new['statusId'] = df_results_new['statusId'].replace([3,4] , "Accident / Collision")
mask = ~df_results_new['statusId'].isin(['Finished',"Accident / Collision"])
df_results_new['statusId'] = df_results_new['statusId'].mask(mask, "Technical Failure")

### Feature: SC Appearance in race
- Binary categorization (Yes/No)

In [48]:
xl = pd.ExcelFile("./formula-1-race-data/cliptheapex.xlsx", encoding='utf-8')
xl.sheet_names
weather = xl.parse("Sheet16")

weather['name'] = weather['name'] + " Grand Prix"
weather.replace("Dy", "Dry", inplace=True)

# Feature: SC Appearance in race
weather['SC Laps'].fillna(0, inplace=True)
weather['SC'] = np.where(weather['SC Laps'] > 0, "SC", "No SC")

### Feature: Pit Stop timings 

In [49]:
df_pitStops = pd.read_csv('./formula-1-race-data/pitStops.csv')
df_pitStops = pd.merge(df_pitStops, dcrm , on=['raceId', 'driverId'], how='left')

#pS_notouliers = remove_outliers(df_pitStops, ['driverRef', 'name', 'year'], 'sum', "milliseconds", 0.9)

### Feature: Count of overtakes in a race per driver

In [116]:
df_overtake = read_from_pickle(directory, "df_overtaking_preprocessed.pickle")
df_overtake.to_csv(directory+'df_overtake.csv', sep='\t', encoding='utf-8')

In [136]:
df_overtake = pd.read_csv(directory+'df_overtaking1517.csv')

## Section C: Build dataset with selected features

### Functions to pre-process data

In [170]:
def remove_outliers(df, groupby_fields, agg_method, col, threshold):
    
    g = df.groupby(groupby_fields)[col].agg(agg_method).reset_index()
    
    # Filter out the outliers
    return g[g[col] < g[col].quantile([0, threshold])[threshold]]


def calc_stats_wrapper(function, df, year_range, col, groupby_field, agg_method, calc_method):
    
    pS_g_all = pd.DataFrame()

    if (calc_method == "rolling_value"):
        
        ranges = [range(year_range[idx]-3, year_range[idx]) for idx,value in enumerate(year_range)]
            
        for r in ranges:
            pS_g = function(df, r, col, groupby_field, agg_method)
            pS_g['year'] = r[-1] + 1
            pS_g_all = pd.concat([pS_g_all, pS_g])
        return pS_g_all
            
    elif (calc_method == 'one_year'):
        
        for r in year_range:
            pS_g = function(df, [r], col, groupby_field, agg_method)
            pS_g['year'] = r
            pS_g_all = pd.concat([pS_g_all, pS_g])
        return pS_g_all
            
    raise ValueError("Only rolling_value and one_year are available options for calc_method")

def calc_stats_btw_grps(df, year_range, col, groupby_field, agg_method):
    
    df = df[df['year'].isin(year_range)] 
    g = df.groupby(groupby_field)[col].agg([agg_method]).reset_index()
    
    if len(groupby_field) > 1:
        df_overall = df.groupby(groupby_field[1:])[col].agg([agg_method]).reset_index()
        df_overall.rename(columns={agg_method: agg_method+' (overall)'}, inplace=True)
        df_new = pd.merge(g, df_overall, on=groupby_field[1:], how='left')
        
        df_new['proportion'] = (df_new[agg_method] / df_new[agg_method+' (overall)'])
        df_new.drop([agg_method, agg_method +' (overall)'], axis=1, inplace=True)
        
        return df_new
        
    elif len(groupby_field) == 1:
        
        total = float(df[col].agg([agg_method])[agg_method])
        for i, row in g.iterrows():
            g.loc[i, 'proportion'] = float(g.loc[i, agg_method]) / total
        
        g.drop([agg_method], axis=1, inplace=True)
            
        return g

def calc_stats(df, year_range, col, groupby_fields, agg_method):
    
    df = df[df['year'].isin(year_range)] 

    g = pd.DataFrame(df.groupby(groupby_fields)[col].value_counts())
    g.rename(columns={col:agg_method}, inplace=True)
    g = g.reset_index()
    
    g_overall = pd.DataFrame(df.groupby(groupby_fields)[col]\
                               .agg(agg_method).rename("total")).reset_index()

    g = pd.merge(g, g_overall, on=groupby_fields, how='left')
    g['percentage'] = (g[agg_method] / g['total']).apply(lambda x: round(x,2))

    gPT = pd.pivot_table(g, index=groupby_fields, columns=[col], values='percentage').reset_index()
    gPT.fillna(0, inplace=True)

    return gPT

### Master function to create dataset containing selected features 

In [234]:
def CREATE_MODEL(TARGET_VAR, year_range, calc_method, rookies, \
                 df_results_new, weather, df_overtake, df_pitStops, \
                 team_cat_dummies, qual_pos, selected_sets1, selected_sets_all):
    
    # =============================================================================================
    # SELECTED FEATURES
    # Feature: StatusId
    status = calc_stats_wrapper(calc_stats, df_results_new, year_range, 'statusId',\
                                    ['name', 'driverRef'], 'count', calc_method)

    # Feature: Finish position
    pos = calc_stats_wrapper(calc_stats, df_results_new, year_range, 'position', \
                             ['name', 'driverRef'], 'count', calc_method)

    # Feature: Safety car
    sc = calc_stats_wrapper(calc_stats, weather, year_range, 'SC', ['name'], 'count', calc_method)

    # Feature: Wet weather rate
    ww = calc_stats_wrapper(calc_stats, weather, year_range, 'weather', ['name'], 'count', calc_method)

    # Feature: Overtaking rate of each driver (Comment out this section even though its a viable features as i do not have the data for the data for the latest season)
    df_overtake = df_overtake[df_overtake['clear lap?'] == 'overtaker']

    o_d = calc_stats_wrapper(calc_stats_btw_grps, df_overtake, year_range, 'clear lap?', ['driverRef'], 'count', calc_method)
    o_d = o_d.rename(columns={'proportion': 'o_p (driver)'})

    # Feature: Overtaking rate of each track
    o_r = calc_stats_wrapper(calc_stats_btw_grps, df_overtake, year_range, 'clear lap?', ['name'], 'count', calc_method)
    o_r = o_r.rename(columns={'proportion': 'o_p (track)'})

    # Feature: Pit stop timings of each team
    pS_d = calc_stats_wrapper(calc_stats_btw_grps, df_pitStops, year_range, 'milliseconds', \
                              ['constructorRef'], 'sum', calc_method)
    pS_d = pS_d.rename(columns={'proportion': 'pS_p (team)'})

    # Feature: Pit stop timings of each team
    pS_notouliers = remove_outliers(df_pitStops, ['driverRef', 'name', 'year'], 'sum', "milliseconds", 0.95)
    pS_r = calc_stats_wrapper(calc_stats_btw_grps, df_pitStops, year_range, 'milliseconds', \
                              ['name'], 'sum', calc_method)
    pS_r = pS_r.rename(columns={'proportion': 'pS_p (track)'})

    # Feature: team (Also use as template/base to merge all the oher features to)
    team_cat_dummies = team_cat_dummies[['year', 'name', 'driverRef', 'constructorRef', \
                                         'team category_middle', 'team category_top']] 
    team_cat_dummies = team_cat_dummies[team_cat_dummies['year'].isin(year_range)]
    
    # =============================================================================================
    # PREPARE FINAL DATAFRAME
    df = pd.merge(team_cat_dummies, status, on=['year','name', 'driverRef'], how='left') 
    
    # Feature: Qualification Position
    df = pd.merge(df, qual_pos, on=['year','name', 'driverRef'], how='left')
    
    # Option 1: Selected Tyre Sets as ordinal categorical vaues
    if (selected_sets_all is None) & (selected_sets1 is not None):
        df = pd.merge(df, selected_sets1, on=['year', 'name', 'driverRef'], how='left') 
    # Option 2: Don't use selected tyre sets as a feature
    elif (selected_sets1 is None) & (selected_sets_all is None):
        df = df
    else:
    # Option 3: Selected Tyre Sets as nominal categorical vaues
        df = pd.merge(df, selected_sets_all, on=['year', 'name', 'driverRef'], how='left')
        
    df = pd.merge(df, pos, on=['year','name', 'driverRef'], how='left')                         
    df = pd.merge(df, sc, on=['year','name'], how='left')
    df = pd.merge(df, ww, on=['year','name'], how='left') 
    df = pd.merge(df, o_d, on=['year', 'driverRef'], how='left') 
    df = pd.merge(df, o_r, on=['year','name'], how='left') 
    df = pd.merge(df, pS_d, on=['year', 'constructorRef'], how='left') 
    df = pd.merge(df, pS_r, on=['year','name'], how='left') 

    # Final step: Merge dataframe containing target variable
    df = pd.merge(df, TARGET_VAR, on=['year', 'name', 'driverRef'], how='left')
    
    # Drop one category amongst each of the feature
    df.drop(['Technical Failure', 'Pos > 10', 'Finished', 'No SC', 'Varied', 'Did not finish'], axis=1, inplace=True)

    # =============================================================================================
    # CLEANING: 
    # Inpute data for rookies
    if calc_method=='rolling_value':
        rookie_stats_all, rookie_o_g = rookie_stats(df_results_new, df_overtake, rookies, year_range)

        df = pd.merge(df, rookie_stats_all, on=['year'], how='left')
        df = pd.merge(df, rookie_o_g, on=['year'], how='left')

        #df['Did not finish'].fillna(df['Average DNF rate'], inplace=True)
        df['Accident / Collision'].fillna(df['Average accident rate'], inplace=True)
        df['Podium'].fillna(df['Average proba of podium finish'], inplace=True)
        df['Pos 4 to 10'].fillna(df['Average proba 4th to 10th pos'], inplace=True)
        df['o_p (driver)'].fillna(df['proportion'], inplace=True)
        df['pS_p (team)'].fillna(0.1, inplace=True)
        
        # Drop the columns only needed for cleaning purposes, now unneccesary
        df.drop(['Average DNF rate', 'Average accident rate', 'Average proba 4th to 10th pos', \
                 'Average proba of podium finish', 'proportion'], axis=1, inplace=True)

    # Inpute data for new tracks
    df['o_p (track)'].fillna(0.05, inplace=True)
    df['pS_p (track)'].fillna(0.05, inplace=True)
    df['SC'].fillna(0.5, inplace=True) 
    df['Dry'].fillna(0.5, inplace=True) 
    df['Wet'].fillna(0.5, inplace=True) 
    
    # Inpute zero for qualifying position for drivers who did not qualify or participate in qualifying
    df['position'].fillna(0, inplace=True)

    df.drop(['constructorRef'], axis=1, inplace=True)

    # Check if there are null values remaining
    print df.isnull().sum()
    
    return df

In [223]:
# Rookies do not have any historical records of their peformance in F1.
# Their data has to be inputed with care as they account for a quarter of drivers in a season.

# Create list of rookies
rookies_2012 = ['grosjean', 'ricciardo', 'vergne', 'pic'] # will have missing values in 2013
rookies_2013 = ['bottas', 'gutierrez', 'bianchi']
rookies_2014 = ['chilton', 'magnussen', 'kvyat', 'ericsson']
rookies_2015 = ['sainz', 'verstappen', 'nasr', 'stevens', 'merhi', 'rossi']
rookies_2016 = ['palmer', 'wehrlein', 'haryanto']
rookies = [rookies_2012, rookies_2013, rookies_2014, rookies_2015, rookies_2016]

def rookie_stats(df_results_new, df_overtake, rookies, year_range):

    acc_list = []
    dnf_list = []
    pos4to10_list = []
    stats = []
    podium_list =[]
    
    acc_avg = []
    dnf_avg = []
    pos4to10_avg = []
    podium_avg = []
    
    years = []
    o_g_all = pd.DataFrame()

    ranges = [range(year_range[idx]-3, year_range[idx]) for idx,value in enumerate(year_range)]
    
    for r,year in zip(rookies, range(year_range[0]-3, year_range[-1])):
        accidents = float(len(df_results_new[(df_results_new['driverRef'].isin(r)) & \
                       (df_results_new['year'] == year) & \
                       (df_results_new['statusId'] == 'Accident / Collision')]))

        dnf = float(len(df_results_new[(df_results_new['driverRef'].isin(r)) & \
                       (df_results_new['year'] == year) & \
                       (df_results_new['position'] == 'Did not finish')]))

        podium = float(len(df_results_new[(df_results_new['driverRef'].isin(r)) & \
                   (df_results_new['year'] == year) & \
                   (df_results_new['position'] == 'Podium')]) )

        pos4to10 = float(len(df_results_new[(df_results_new['driverRef'].isin(r)) & \
                   (df_results_new['year'] == year) & \
                   (df_results_new['position'] == 'Pos 4 to 10')]) )

        total = float(len(df_results_new[(df_results_new['driverRef'].isin(r)) & \
                       (df_results_new['year']== year)]))

        acc_list.append(accidents/total)
        dnf_list.append(dnf/total)
        pos4to10_list.append(pos4to10/total)
        podium_list.append(podium/total)
    
    df_overtake = df_overtake[df_overtake['clear lap?'] == 'overtaker']   
    o = calc_stats_wrapper(calc_stats_btw_grps, df_overtake, year_range, 
                             'clear lap?',['driverRef'], 'count', 'rolling_value')
        
    # Only select rookie drivers
    for r, d in zip(year_range, rookies):
        o_new = o[(o['driverRef'].isin(d)) & (o['year'] == r)]
        o_g = o_new.groupby(['year']).agg({'proportion': 'mean'}).reset_index()
        o_g_all = pd.concat([o_g_all,  o_g])

    for idx,r in enumerate(ranges):
        acc_avg.append(round(np.mean(list(np.array(acc_list)[range(idx, idx+3)])), 2))
        dnf_avg.append(round(np.mean(list(np.array(dnf_list)[range(idx, idx+3)])), 2))
        pos4to10_avg.append(round(np.mean(list(np.array(pos4to10_list)[range(idx, idx+3)])), 2))
        podium_avg.append(round(np.mean(list(np.array(podium_list)[range(idx, idx+3)])), 2))
        years.append(r[-1] + 1)

    stats = pd.DataFrame({"Average accident rate": acc_avg,
                          'Average DNF rate': dnf_avg,
                          'Average proba 4th to 10th pos': pos4to10_avg,
                          'Average proba of podium finish': podium_avg,
                          'year': years})

    return stats, o_g_all


def find_in_list_of_list(mylist, char):
    for sub_list in mylist:
        if char in sub_list:
            return (mylist.index(sub_list), sub_list.index(char))
    raise ValueError("'{char}' is not in list".format(char = char))
    

# Section D

## 1) Binary Classification: Race Finishes

The problem is supervised binary class classification problem, and our goal is to predict whether or not a driver finishes a race. Each driver can only be assigned to one category. There are 20 races in a Formula 1 race calendar, so we have to make predictions for each of the races as the season progresses.

### 1a) Create dataset of features

#### Summary of features:
1. Team category_middle: Driver belongs to a mid-field team (Binary category)
2. Team category_top: Driver belongs to a top team (Binary category)
3. Accident / Collision: Accident / Collision rate at each race per driver (Continuous variable)
4. Position: Qualifying postion of driver at each race (Ordinal variable)
5. Podium: Did driver have a podium finish?(ie. finish in the top 3 position) (Binary category)
6. Pos 4 to 10: Did driver finish in the top 4 to 10 position?   (Binary category)
7. SC: Did the Safety Car appear at least once during the race?  (Binary category)
8. Dry: Was the track dry during the race?  (Binary category)
9. Wet: Was the weather rainy during the race? (Binary category)
10. o_p (driver): Number of overtakes for each DRIVER as a proportion of the total number of overtakes in the season (Continuous variable)
11. o_p (track): Number of overtakes for each TRACK as a proportion of the total number of overtakes in the season (Continuous variable)
12. pS_p (team): Average Pit Stop Timing (time to complete a pitstop) for each TEAM as a proportion of the total pitstop times in the season (Continuous variable)
13. pS_p (track): Average Pit Stop Timing for each TRACK as a proportion of the total pitstop times in the season Continuous variable)

#### Note:
- For all of the features except 'team category' and 'qualifying position', we do not actually know the true values of these features pre-race for the race that we are predicting on. Hence, these values are <b>estimated</b> by average the rate from the last 3 years. 

In [68]:
status = df_results_new[['year', 'name', 'driverRef', 'statusId']]
status.replace('Finished', 1, inplace=True)
status.replace('Accident / Collision', 0, inplace=True)
status.replace('Technical Failure', 0, inplace=True)

In [228]:
status_dataset_no_agg = CREATE_MODEL(status, [2015, 2016, 2017], 'one_year', rookies, \
                                    df_results_new, weather, df_overtake, df_pitStops, \
                                    team_cat_dummies, qual_pos, selected_sets1=None, selected_sets_all=None)

year                     0
name                     0
driverRef                0
team category_middle     0
team category_top        0
Accident / Collision    11
position                 0
Podium                  11
Pos 4 to 10             11
SC                       0
Dry                      0
Wet                      0
o_p (driver)             9
o_p (track)              0
pS_p (team)              0
pS_p (track)             0
statusId                11
dtype: int64


In [229]:
status_dataset_no_agg = status_dataset_no_agg.dropna(how='any',axis=0)
status_dataset_no_agg.to_pickle(os.path.join(directory,  "status_dataset_no_agg.pickle"))

In [238]:
status_dataset = CREATE_MODEL(status, [2016, 2017], 'rolling_value', rookies, \
                             df_results_new, weather, df_overtake, df_pitStops, \
                             team_cat_dummies, qual_pos, selected_sets1, selected_sets_all=None)

year                    0
name                    0
driverRef               0
team category_middle    0
team category_top       0
Accident / Collision    0
position                0
Medium                  5
Soft                    5
Super Soft              5
Ultra soft              5
Hard                    5
Podium                  0
Pos 4 to 10             0
SC                      0
Dry                     0
Wet                     0
o_p (driver)            0
o_p (track)             0
pS_p (team)             0
pS_p (track)            0
statusId                9
dtype: int64


In [249]:
dataset = pd.concat([train_set, test_set])
dataset.to_csv(directory+'dataset.csv', sep='\t')
status_dataset.to_pickle(os.path.join(directory,  "status_dataset.pickle"))